In [1]:
%load_ext autoreload   

In [2]:
k = 1000
model = 'gpt-4o-mini'

In [66]:
%load_ext autoreload
%autoreload 2

import asyncio
import pandas as pd
from meeting_search import MeetingSearch, build_context_string
from typing import List

from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens

from vexa import VexaAPI
from core import system_msg, user_msg,generic_call_stream
from datetime import datetime

from prompts import Prompts
prompts = Prompts()

analyzer = MeetingSearch()
print(analyzer.count_documents())
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens, assistant_msg
from pydantic_models import QueryPlan, Query


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1279


In [73]:
async def generate_output(query, messages):
    query_ = ' '.join([m.content for m in messages])+' '+query
    queries = await analyzer.generate_search_queries(query_)
    print(queries)
    summaries =  analyzer.get_summaries()
    full_context, meeting_ids = await analyzer.build_context(queries, summaries,include_all_summaries=False)
    pref = f"Based on the following context, answer the question:" if len(messages) == 0 else "Update your previous answer given the follow-up request:"
    messages_context = [system_msg(prompts.perplexity), user_msg(f"Context:\n{full_context}")] + messages + [user_msg(f"{pref} {query}")]
    output = await generic_call_stream(messages_context)
    messages.append(user_msg(query))
    messages.append(assistant_msg(output))
    return output, messages



In [74]:
messages=[]

In [75]:
query = 'list seo meetings'

In [76]:
output, messages = await generate_output(query, messages)

[{'start': None, 'end': None, 'vector_search_query': 'SEO strategies and content generation discussion'}, {'start': None, 'end': None, 'vector_search_query': 'SEO tool development discussion'}, {'start': None, 'end': None, 'vector_search_query': 'SEO strategies discussion'}, {'start': None, 'end': None, 'vector_search_query': 'Google Ads campaign review and strategy'}, {'start': None, 'end': None, 'vector_search_query': 'User feedback and outreach strategy discussion'}]
The SEO meetings discussed in the context are as follows:

1. **SEO Marketing Strategy Discussion (2024-09-06 17:32)**: This meeting focused on competitor SEO marketing strategies and Google Ads, emphasizing the need to review initial ideas and develop onboarding processes. The importance of interpreting data from SimilarWeb to inform marketing decisions was also highlighted[4].

2. **SEO Tool Development Discussion (2024-10-01 12:19)**: In this meeting, Rick Tousseyn and Dmitriy Grankin discussed a content creation too

In [78]:
output, messages = await generate_output('wheech tools mentioned?', messages)

[{'start': None, 'end': None, 'vector_search_query': 'SEO marketing strategies and Google Ads, competitor analysis, onboarding processes, SimilarWeb data interpretation.'}, {'start': None, 'end': None, 'vector_search_query': 'content creation tool for SEO optimization, keyword targeting, article ranking strategies, collaboration potential, case studies.'}, {'start': None, 'end': None, 'vector_search_query': 'improving SEO and content generation, collaboration with SEO experts, updating articles, AI in content creation.'}]
The following tools were mentioned in the SEO meetings:

1. **SimilarWeb**: Used for interpreting data to inform marketing decisions and analyze competitors' strategies[4].

2. **Surfer SEO**: Discussed for its content editor features, which help optimize articles based on keyword targeting and competitor analysis. It provides a content score and recommendations for improving SEO performance[10][11].

3. **Rank Math**: An SEO plugin for WordPress that helps manage met

In [79]:
output, messages = await generate_output('what is Yoast?', messages)


[{'start': datetime.datetime(2024, 9, 6, 17, 32, tzinfo=TzInfo(UTC)), 'end': datetime.datetime(2024, 9, 6, 17, 32, tzinfo=TzInfo(UTC)), 'vector_search_query': 'SEO Marketing Strategy Discussion'}, {'start': datetime.datetime(2024, 10, 1, 12, 19, tzinfo=TzInfo(UTC)), 'end': datetime.datetime(2024, 10, 1, 12, 19, tzinfo=TzInfo(UTC)), 'vector_search_query': 'SEO Tool Development Discussion'}, {'start': datetime.datetime(2024, 10, 3, 12, 0, tzinfo=TzInfo(UTC)), 'end': datetime.datetime(2024, 10, 3, 12, 0, tzinfo=TzInfo(UTC)), 'vector_search_query': 'SEO Strategies and Content Generation Discussion'}, {'start': None, 'end': None, 'vector_search_query': 'SimilarWeb'}, {'start': None, 'end': None, 'vector_search_query': 'Surfer SEO'}, {'start': None, 'end': None, 'vector_search_query': 'Rank Math'}, {'start': None, 'end': None, 'vector_search_query': 'Yoast'}, {'start': None, 'end': None, 'vector_search_query': 'Google Search Console'}, {'start': None, 'end': None, 'vector_search_query': 'Pag

In [81]:
messages

[Msg(role='user', content='list seo meetings', stage=None),
 Msg(role='assistant', content="The SEO meetings discussed in the context are as follows:\n\n1. **SEO Marketing Strategy Discussion (2024-09-06 17:32)**: This meeting focused on competitor SEO marketing strategies and Google Ads, emphasizing the need to review initial ideas and develop onboarding processes. The importance of interpreting data from SimilarWeb to inform marketing decisions was also highlighted[4].\n\n2. **SEO Tool Development Discussion (2024-10-01 12:19)**: In this meeting, Rick Tousseyn and Dmitriy Grankin discussed a content creation tool for SEO optimization, exploring existing tools, keyword targeting, and strategies for improving article rankings. They emphasized collaboration potential and the importance of case studies in demonstrating the tool's effectiveness[11].\n\n3. **SEO Strategies and Content Generation Discussion (2024-10-03 12:00)**: This meeting involved strategies for improving SEO and content

In [63]:
output

"The SEO meetings discussed in the context are as follows:\n\n1. **SEO Marketing Strategy Discussion (2024-09-06 17:32)**: This meeting focused on competitor SEO marketing strategies and Google Ads, emphasizing the need to review initial ideas and develop onboarding processes. The importance of interpreting data from SimilarWeb to inform marketing decisions was also highlighted[4].\n\n2. **SEO Tool Development Discussion (2024-10-01 12:19)**: In this meeting, Rick Tousseyn and Dmitriy Grankin discussed a content creation tool for SEO optimization, exploring existing tools, keyword targeting, and strategies for improving article rankings. They emphasized collaboration potential and the importance of case studies in demonstrating the tool's effectiveness[8].\n\n3. **SEO Strategies and Content Generation Discussion (2024-10-03 12:00)**: This meeting involved strategies for improving SEO and content generation, with insights shared on updating outdated articles and the potential of AI in c

In [53]:
' '.join([m.content for m in messages])
queries = await analyzer.generate_search_queries(output+' '+query)

['find all meetings that where about seo',
 "The meetings that discussed SEO are as follows:\n\n1. **SEO Marketing Strategy Discussion (2024-09-06 17:32)**: Olga Nemirovskaya and Dmitriy Grankin discussed competitor SEO marketing strategies and Google Ads, emphasizing the need to review initial ideas and develop onboarding processes. They highlighted the importance of interpreting data from SimilarWeb to inform marketing decisions[4].\n\n2. **SEO Tool Development Discussion (2024-10-01 12:19)**: Rick Tousseyn and Dmitriy Grankin discussed a content creation tool for SEO optimization, exploring existing tools, keyword targeting, and strategies for improving article rankings. They emphasized collaboration potential and the importance of case studies in demonstrating the tool's effectiveness[8].\n\n3. **SEO Strategies and Content Generation Discussion (2024-10-03 12:00)**: Olga Mykhoparkina and Dmitriy Grankin discussed strategies for improving SEO and content generation, including collab

In [51]:
queries = await analyzer.generate_search_queries(output+' '+query)
print(queries)

[{'start': None, 'end': None, 'vector_search_query': 'SEO marketing strategies and analytics tools discussed in meetings, including SimilarWeb and content creation tools for SEO optimization.'}, {'start': None, 'end': None, 'vector_search_query': 'Discussions on SEO strategies, content generation, and analytics tools used for improving article rankings and marketing decisions.'}, {'start': None, 'end': None, 'vector_search_query': 'Meetings that focused on SEO, including tools for keyword targeting and strategies for enhancing content visibility.'}, {'start': None, 'end': None, 'vector_search_query': 'SEO discussions involving analytics tools and strategies for optimizing content and improving search engine rankings.'}, {'start': None, 'end': None, 'vector_search_query': 'Meetings that covered SEO strategies, including the use of analytics tools like SimilarWeb and content creation tools for optimization.'}]


In [47]:
messages

[Msg(role='user', content='find all meetings that where about seo', stage=None),
 Msg(role='user', content="The meetings that discussed SEO are as follows:\n\n1. **SEO Marketing Strategy Discussion (2024-09-06 17:32)**: Olga Nemirovskaya and Dmitriy Grankin discussed competitor SEO marketing strategies and Google Ads, emphasizing the need to review initial ideas and develop onboarding processes. They highlighted the importance of interpreting data from SimilarWeb to inform marketing decisions[4].\n\n2. **SEO Tool Development Discussion (2024-10-01 12:19)**: Rick Tousseyn and Dmitriy Grankin discussed a content creation tool for SEO optimization, exploring existing tools, keyword targeting, and strategies for improving article rankings. They emphasized collaboration potential and the importance of case studies in demonstrating the tool's effectiveness[8].\n\n3. **SEO Strategies and Content Generation Discussion (2024-10-03 12:00)**: Olga Mykhoparkina and Dmitriy Grankin discussed strate

In [ ]:
await generate_output('which analityc tools mentioned?', messages)

In [16]:

meeting_id_ref = 9
dashboard_url = f"https://dashboard.vexa.ai/#{meeting_ids[meeting_id_ref-1]}"
dashboard_url


'https://dashboard.vexa.ai/#6cf0b989-5e9c-4a57-a058-8f6ac0ca8025'